# L1 – Captura, limpieza e ingeniería de características

Este notebook construye el **Dataset v1 limpio (E-1)** y el **dataset de features de la cohorte L1**, a partir de `clientify_contactos.csv` exportado de Clientify.

In [2]:
# Imports y rutas

import os
import pandas as pd

pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", 20)

RAW_PATH = os.path.join("..", "data", "raw", "clientify_contactos.csv")
CLEAN_PATH = os.path.join("..", "data", "processed", "l1_clean.csv")
FEATURES_PATH = os.path.join("..", "data", "processed", "l1_features.csv")

In [4]:
# Cargar datos crudos

df_raw = pd.read_csv(RAW_PATH, encoding="latin-1")
df_raw.shape, df_raw.head()

((224, 24),
           ID       nombre         apellidos empresa               propietario  \
 0  122401317        Sandra           Medina     NaN  Uriel Gerónimo Velázquez   
 1  123064934   LUIS CARLOS  LEAL VILLANUEVA     NaN  Uriel Gerónimo Velázquez   
 2  122504940      Georgina            Lopez     NaN  Uriel Gerónimo Velázquez   
 3  122701333  José Antonio     Pérez Macedo     NaN  Uriel Gerónimo Velázquez   
 4  122645644     Monserrat         Estrella     NaN  Uriel Gerónimo Velázquez   
 
    origen    medium        referrer utm source  utm medium  \
 0     NaN   unknown         bit.ly/         FB   ROITiempo   
 1     NaN  referido          bit.ly         FB   ROITiempo   
 2     NaN   unknown  bit.ly/4l3M30o         FB  dolorExcel   
 3     NaN    social        Facebook         FB  dolorExcel   
 4     NaN  referido          bit.ly         FB  dolorExcel   
 
             utm campaign utm content                    utm term   creado  \
 0  MasterData14-AGO25-V1   ROITiemp

In [5]:
# Normalizar nombres de columnas

df = df_raw.copy()

df.columns = (
    df.columns
    .str.strip()
    .str.lower()
    .str.replace(" ", "_")
    .str.replace("ú", "u")
    .str.replace("ó", "o")
    .str.replace("í", "i")
    .str.replace("é", "e")
    .str.replace("á", "a")
)

df.columns.tolist()

['id',
 'nombre',
 'apellidos',
 'empresa',
 'propietario',
 'origen',
 'medium',
 'referrer',
 'utm_source',
 'utm_medium',
 'utm_campaign',
 'utm_content',
 'utm_term',
 'creado',
 'ultimo_contacto',
 'status',
 'gdpr_accept',
 'lead_scoring',
 'observaciones',
 'email_1',
 'telefono_1',
 'telefono_2',
 'etapa_de_oportunidad_activa',
 'asistio']

In [6]:
# Crear target_asiste y limpiar básicos

# Limpiar texto en columnas clave si existen
for col in ["nombre_", "apellidos", "empresa", "propietario"]:
    if col in df.columns:
        df[col] = df[col].astype(str).str.strip()

# Normalizar 'asistio'
if "asistio" not in df.columns:
    raise ValueError("No se encontró la columna 'asistio'.")

df["asistio"] = df["asistio"].fillna("No")
df["asistio_limpio"] = df["asistio"].astype(str).str.strip().str.lower()

df["target_asiste"] = (df["asistio_limpio"] == "si").astype(int)
df["target_asiste"].value_counts()

target_asiste
0    188
1     36
Name: count, dtype: int64

In [7]:
# Tipos y duplicados

# lead_scoring
if "lead_scoring" in df.columns:
    df["lead_scoring"] = pd.to_numeric(df["lead_scoring"], errors="coerce").fillna(0).astype(int)

# gdpr_accept
if "gdpr_accept" in df.columns:
    df["gdpr_accept"] = df["gdpr_accept"].astype(str).str.lower().isin(["true", "1"])

# Teléfonos como texto
for col in ["telefono_1", "telefono_2"]:
    if col in df.columns:
        df[col] = df[col].astype(str).str.replace(".0", "", regex=False)

# email_1 -> email
if "email_1" in df.columns and "email" not in df.columns:
    df = df.rename(columns={"email_1": "email"})

# Eliminar duplicados por email
if "email" in df.columns:
    before = len(df)
    df = df.drop_duplicates(subset=["email"], keep="first")
    after = len(df)
    print(f"Duplicados eliminados por email: {before - after}")
else:
    print("No se encontró columna 'email'.")

df.shape

Duplicados eliminados por email: 0


(224, 26)

In [8]:
# Guardar Dataset v1 limpio

os.makedirs(os.path.dirname(CLEAN_PATH), exist_ok=True)
df.to_csv(CLEAN_PATH, index=False, encoding="utf-8")
CLEAN_PATH

'../data/processed/l1_clean.csv'

## 2. Ingeniería de características (L1)

In [9]:
# Leer limpio

df_clean = pd.read_csv(CLEAN_PATH, encoding="utf-8")
df_clean.shape

(224, 26)

In [10]:
# Función para creado

def parse_creado_to_seconds(creado_str):
    if pd.isna(creado_str):
        return None
    try:
        minutos, resto = str(creado_str).split(":")
        segundos = float(resto)
        return int(minutos) * 60 + segundos
    except Exception:
        return None

In [11]:
# Construcción de features

df_feat = df_clean.copy()

# Tiempo a partir de 'creado'
if "creado" in df_feat.columns:
    df_feat["creado_segundos"] = df_feat["creado"].apply(parse_creado_to_seconds)

# medium flags
if "medium" in df_feat.columns:
    df_feat["medium_lower"] = df_feat["medium"].astype(str).str.lower()
    df_feat["f_medium_social"] = (df_feat["medium_lower"] == "social").astype(int)
    df_feat["f_medium_referido"] = (df_feat["medium_lower"] == "referido").astype(int)
    df_feat["f_medium_unknown"] = (df_feat["medium_lower"] == "unknown").astype(int)
    df_feat["f_medium_directo"] = (df_feat["medium_lower"] == "directo").astype(int)

# teléfono 2
if "telefono_2" in df_feat.columns:
    df_feat["f_tiene_telefono_2"] = df_feat["telefono_2"].notna().astype(int)

# GDPR
if "gdpr_accept" in df_feat.columns:
    df_feat["f_gdpr_accept_int"] = df_feat["gdpr_accept"].astype(int)

# longitudes
if "nombre_" in df_feat.columns:
    df_feat["f_len_nombre"] = df_feat["nombre_"].astype(str).str.len()

if "empresa" in df_feat.columns:
    df_feat["f_len_empresa"] = df_feat["empresa"].astype(str).str.len()

if "target_asiste" not in df_feat.columns:
    raise ValueError("No se encontró 'target_asiste'.")

In [12]:
feature_cols = [
    "creado_segundos",
    "lead_scoring",
    "f_medium_social",
    "f_medium_referido",
    "f_medium_unknown",
    "f_medium_directo",
    "f_tiene_telefono_2",
    "f_gdpr_accept_int",
    "f_len_nombre",
    "f_len_empresa",
    "medium",
    "utm_source",
    "utm_medium",
    "utm_campaign",
    "utm_content",
    "utm_term",
    "status",
    "etapa_de_oportunidad_activa",
    "propietario",
    "origen",
]

feature_cols = [c for c in feature_cols if c in df_feat.columns]
print(f"Total columnas de features seleccionadas: {len(feature_cols)}")

df_features = df_feat[feature_cols + ["target_asiste"]].copy()
df_features.head()

Total columnas de features seleccionadas: 19


,creado_segundos,lead_scoring,f_medium_social,f_medium_referido,f_medium_unknown,f_medium_directo,f_tiene_telefono_2,f_gdpr_accept_int,f_len_empresa,medium,utm_source,utm_medium,utm_campaign,utm_content,utm_term,status,etapa_de_oportunidad_activa,propietario,origen,target_asiste
0,2086.8,0,0,0,1,0,0,0,3,unknown,FB,ROITiempo,MasterData14-AGO25-V1,ROITiempo,SegmentoAnalistaFinanciero,en oportunidad,Conversación Iniciada,Uriel Gerónimo Velázquez,NaN,0
1,1357.6,0,0,1,0,0,0,0,3,referido,FB,ROITiempo,MasterData14-AGO25-V1,ROITiempo,SegmentoAnalistaFinanciero,en oportunidad,Registro,Uriel Gerónimo Velázquez,NaN,0
2,288.2,0,0,0,1,0,0,0,3,unknown,FB,dolorExcel,MasterData14-AGO25-V1,dolorExcel,SegmentoAnalistaFinanciero,en oportunidad,Confirmación a S1,Uriel Gerónimo Velázquez,NaN,0
3,2174.0,0,1,0,0,0,0,0,3,social,FB,dolorExcel,MasterData14-AGO25-V1,dolorExcel,SegmentoAnalistaFinanciero,en oportunidad,Confirmación a S1,Uriel Gerónimo Velázquez,NaN,0
4,414.2,0,0,1,0,0,1,0,3,referido,FB,dolorExcel,MasterData14-AGO25-V1,dolorExcel,SegmentoLibre,en oportunidad,Conversación Iniciada,Uriel Gerónimo Velázquez,NaN,0


In [13]:
# Guardar dataset de features

os.makedirs(os.path.dirname(FEATURES_PATH), exist_ok=True)
df_features.to_csv(FEATURES_PATH, index=False, encoding="utf-8")
FEATURES_PATH

'../data/processed/l1_features.csv'

In [14]:
# Resumen

print("Shape Dataset v1 limpio:", df_clean.shape)
print("Shape Dataset features L1:", df_features.shape)
print("\nDistribución target_asiste:")
print(df_features["target_asiste"].value_counts())

Shape Dataset v1 limpio: (224, 26)
Shape Dataset features L1: (224, 20)

Distribución target_asiste:
target_asiste
0    188
1     36
Name: count, dtype: int64
